In [101]:
import pandas as pd
import numpy as np
import datetime
import math
import sklearn.cross_validation
from scipy import stats

In [102]:
train = pd.read_csv('data_original/train.csv')
test = pd.read_csv('data_original/test.csv')
    
frame = [train, test]
common = pd.concat(frame)
common.Date = common.Date.apply(lambda x: pd.to_datetime(x))
common = common.sort_values(by=['Date'])

common.to_csv('data_original/common.csv', index=None)

# date.csv

In [103]:
date = pd.read_csv('data_original/common.csv')[['Date']]
date.Date = date.Date.apply(lambda x: pd.to_datetime(x))

In [104]:
start_date = date.iloc[0]['Date']
date['Year'] = date.Date.apply(lambda x: x.year)
date['Month'] = date.Date.apply(lambda x: x.month)
date['MonthDay'] = date.Date.apply(lambda x: x.day)
date['WeekDay'] = date.Date.apply(lambda x: x.weekday())
date['Point'] = pd.Series(common.Point.as_matrix())

In [105]:
date['YearDay'] = date.Date.apply(lambda x: (x-pd.to_datetime(str(x.year)+'-01-01')).days)

In [106]:
date['ThroughMonth'] = date.Date.apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)

In [107]:
date['ThroughWeek'] = date.Date.apply(lambda x: int( ( (x - start_date).days + start_date.weekday()) / 7))

In [108]:
date['ThroughDay'] = date.Date.apply(lambda x: (x - start_date).days)

In [109]:
date.to_csv('data_transform/date.csv', index=None)

In [110]:
date.head(10)

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay
0,2011-10-06,2011,10,6,3,���������-1,278,0,0,0
1,2011-10-07,2011,10,7,4,���������-1,279,0,0,1
2,2011-10-09,2011,10,9,6,���������-1,281,0,0,3
3,2011-11-15,2011,11,15,1,���������-1,318,1,6,40
4,2011-12-08,2011,12,8,3,���������-1,341,2,9,63
5,2011-12-09,2011,12,9,4,���������-1,342,2,9,64
6,2011-12-10,2011,12,10,5,���������-1,343,2,9,65
7,2011-12-11,2011,12,11,6,���������-1,344,2,9,66
8,2011-12-13,2011,12,13,1,���������-1,346,2,10,68
9,2011-12-14,2011,12,14,2,���������-1,347,2,10,69


# target.csv

In [117]:
target = pd.read_csv('data_original/common.csv')[['Count', 'Date']]
target['IsTrain'] = pd.Series(np.array([1]*train.shape[0] + [0]*test.shape[0]))
target['IsTest'] = pd.Series(np.array([0]*train.shape[0] + [1]*test.shape[0]))

In [118]:
target.to_csv('data_transform/target.csv', index = None)

In [119]:
target.head(10)

,Count,Date,IsTrain,IsTest
0,8,2011-10-06,1,0
1,88,2011-10-07,1,0
2,1,2011-10-09,1,0
3,1,2011-11-15,1,0
4,2,2011-12-08,1,0
5,43,2011-12-09,1,0
6,10,2011-12-10,1,0
7,1,2011-12-11,1,0
8,60,2011-12-13,1,0
9,83,2011-12-14,1,0


In [120]:
target.tail(10)

,Count,Date,IsTrain,IsTest
53431,66,2016-10-19,0,1
53432,69,2016-10-19,0,1
53433,30,2016-10-19,0,1
53434,94,2016-10-19,0,1
53435,39,2016-10-19,0,1
53436,69,2016-10-19,0,1
53437,76,2016-10-19,0,1
53438,125,2016-10-19,0,1
53439,130,2016-10-19,0,1
53440,97,2016-10-19,0,1


# city.csv


In [121]:
cities = pd.read_csv('data_original/common.csv')[['Point']].Point.unique()
city = pd.DataFrame()
city['Point'] = pd.Series(cities)
city['CityName'] = city.Point.apply(lambda x: '-'.join(x.split('-')[0:-1]))
city_names = city.CityName.unique()
city['CityID'] = city.CityName.apply(lambda x: np.where(city_names == x)[0][0]) 
city['BranchNumber'] = city.Point.apply(lambda x: x.split('-')[-1:][0])
# Москва 1 -> Москва
city['CityName'] = city.CityName.apply(lambda x: ('Москва' if x == 'Москва 1' else x))
city.to_csv('data_transform/city.csv', index = None)

In [122]:
city.head(10)

,Point,CityName,CityID,BranchNumber
0,���������-1,���������,0,1
1,���������-2,���������,0,2
2,����-1,����,1,1
3,��������-1,��������,2,1
4,�������-1,�������,3,1
5,������-1,������,4,1
6,���������-1,���������,5,1
7,��������-1,��������,6,1
8,��������-2,��������,2,2
9,������-2,������,4,2


# internal_features.csv

In [197]:
src = pd.read_csv('data_transform/date.csv')
src['Point'] = pd.read_csv('data_original/common.csv')['Point']


tmp = target.copy()
tmp.Count.loc[tmp.IsTest == 1] = None
withNaNValue = tmp.Count
src['Count'] = withNaNValue

dst = pd.DataFrame() 

In [198]:
src.head()

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay,Count
0,2011-10-06,2011,10,6,3,���������-1,278,0,0,0,8.0
1,2011-10-07,2011,10,7,4,���������-1,279,0,0,1,88.0
2,2011-10-09,2011,10,9,6,���������-1,281,0,0,3,1.0
3,2011-11-15,2011,11,15,1,���������-1,318,1,6,40,1.0
4,2011-12-08,2011,12,8,3,���������-1,341,2,9,63,2.0


In [196]:
src.tail()

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay,Count
53436,2016-10-19,2016,10,19,2,������ 1-1,292,60,263,1840,NaN
53437,2016-10-19,2016,10,19,2,�����������-1,292,60,263,1840,NaN
53438,2016-10-19,2016,10,19,2,������������-1,292,60,263,1840,NaN
53439,2016-10-19,2016,10,19,2,���������-1,292,60,263,1840,NaN
53440,2016-10-19,2016,10,19,2,�����������-1,292,60,263,1840,NaN


## Mean

In [179]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point', 'ThroughDay'])
need_mean_list

Index([u'Month', u'MonthDay', u'ThroughMonth', u'ThroughWeek', u'WeekDay',
       u'Year', u'YearDay'],
      dtype='object')

### by date independed from points

In [180]:
def calc_mean(src, group_column, target_column):
    mean = src.groupby([group_column])[target_column].mean()
    return src[group_column].apply(lambda x: mean[x])

In [181]:

for column in need_mean_list:
    dst['MeanFor' + column] = calc_mean(src, column, 'Count')

In [182]:
dst.head(15)

,MeanForMonth,MeanForMonthDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay
0,105.106277,105.396468,32.333333,32.333333,91.300923,85.111111,83.218391
1,105.106277,104.752427,32.333333,32.333333,122.559729,85.111111,80.670455
2,105.106277,105.499093,32.333333,32.333333,122.119897,85.111111,105.079545
3,103.791141,102.446556,1.000000,1.000000,86.350693,85.111111,125.969697
4,104.978698,107.024242,95.652174,14.000000,91.300923,85.111111,75.836538
5,104.978698,105.499093,95.652174,14.000000,122.559729,85.111111,76.759615
6,104.978698,104.627037,95.652174,14.000000,141.221922,85.111111,83.884615
7,104.978698,100.734055,95.652174,14.000000,122.119897,85.111111,101.800000
8,104.978698,105.141572,95.652174,97.500000,86.350693,85.111111,127.679245
9,104.978698,104.057143,95.652174,97.500000,87.106548,85.111111,92.768519


### by date depended from points

In [183]:
def map_group_two_key(group, x, y):
    return group[x][y]

def calc_mean_by_columns(src, group_columns, target_column):
    mean = src.groupby(group_columns)[target_column].mean()
    return src[group_columns].apply(lambda x: map_group_two_key(mean, *x), axis=1)



In [184]:
for column in need_mean_list:
    dst['MeanFor' + column + 'ByPoints'] = calc_mean_by_columns(src, [column, 'Point'], 'Count')

In [185]:
dst['Point'] = pd.read_csv('data_original/common.csv')['Point']
dst.to_csv('data_transform/internal_features.csv', index=None)

In [186]:
dst.head(15)

,MeanForMonth,MeanForMonthDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints,Point
0,105.106277,105.396468,32.333333,32.333333,91.300923,85.111111,83.218391,267.076336,236.551724,32.333333,32.333333,212.893281,85.111111,220.4,���������-1
1,105.106277,104.752427,32.333333,32.333333,122.559729,85.111111,80.670455,267.076336,244.741379,32.333333,32.333333,290.403162,85.111111,230.8,���������-1
2,105.106277,105.499093,32.333333,32.333333,122.119897,85.111111,105.079545,267.076336,242.694915,32.333333,32.333333,266.624506,85.111111,204.2,���������-1
3,103.791141,102.446556,1.000000,1.000000,86.350693,85.111111,125.969697,258.991736,228.728814,1.000000,1.000000,194.561265,85.111111,208.2,���������-1
4,104.978698,107.024242,95.652174,14.000000,91.300923,85.111111,75.836538,235.299320,255.620690,95.652174,14.000000,212.893281,85.111111,182.6,���������-1
5,104.978698,105.499093,95.652174,14.000000,122.559729,85.111111,76.759615,235.299320,242.694915,95.652174,14.000000,290.403162,85.111111,183.8,���������-1
6,104.978698,104.627037,95.652174,14.000000,141.221922,85.111111,83.884615,235.299320,230.258621,95.652174,14.000000,311.138889,85.111111,181.6,���������-1
7,104.978698,100.734055,95.652174,14.000000,122.119897,85.111111,101.800000,235.299320,230.034483,95.652174,14.000000,266.624506,85.111111,174.4,���������-1
8,104.978698,105.141572,95.652174,97.500000,86.350693,85.111111,127.679245,235.299320,229.465517,95.652174,97.500000,194.561265,85.111111,214.0,���������-1
9,104.978698,104.057143,95.652174,97.500000,87.106548,85.111111,92.768519,235.299320,236.948276,95.652174,97.500000,202.063745,85.111111,225.6,���������-1


# All in one

In [187]:
data = pd.read_csv('data_transform/target.csv')
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
internal_features = pd.read_csv('data_transform/internal_features.csv')



need_columns = date.columns
data[need_columns] = date[need_columns]

need_columns = internal_features.columns
data[need_columns] = internal_features[need_columns]

data.to_csv('data_transform/all.csv', index=None)

In [200]:
data.head()

,Count,Date,IsTrain,IsTest,Year,Month,MonthDay,WeekDay,Point,YearDay,...,MeanForWeekDay,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,8,2011-10-06,1,0,2011,10,6,3,���������-1,278,...,91.300923,85.111111,83.218391,267.076336,236.551724,32.333333,32.333333,212.893281,85.111111,220.4
1,88,2011-10-07,1,0,2011,10,7,4,���������-1,279,...,122.559729,85.111111,80.670455,267.076336,244.741379,32.333333,32.333333,290.403162,85.111111,230.8
2,1,2011-10-09,1,0,2011,10,9,6,���������-1,281,...,122.119897,85.111111,105.079545,267.076336,242.694915,32.333333,32.333333,266.624506,85.111111,204.2
3,1,2011-11-15,1,0,2011,11,15,1,���������-1,318,...,86.350693,85.111111,125.969697,258.991736,228.728814,1.000000,1.000000,194.561265,85.111111,208.2
4,2,2011-12-08,1,0,2011,12,8,3,���������-1,341,...,91.300923,85.111111,75.836538,235.299320,255.620690,95.652174,14.000000,212.893281,85.111111,182.6


In [201]:
data.tail()

,Count,Date,IsTrain,IsTest,Year,Month,MonthDay,WeekDay,Point,YearDay,...,MeanForWeekDay,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
53436,69,2016-10-19,0,1,2016,10,19,2,������ 1-1,292,...,87.106548,111.106047,82.545455,127.000000,90.888889,127.00,NaN,78.850000,104.054152,NaN
53437,76,2016-10-19,0,1,2016,10,19,2,�����������-1,292,...,87.106548,111.106047,82.545455,86.378788,84.000000,88.75,NaN,73.458333,94.816547,76.0
53438,125,2016-10-19,0,1,2016,10,19,2,������������-1,292,...,87.106548,111.106047,82.545455,95.285714,111.470588,172.25,NaN,95.459459,146.812950,79.0
53439,130,2016-10-19,0,1,2016,10,19,2,���������-1,292,...,87.106548,111.106047,82.545455,112.200000,117.388889,188.50,NaN,98.536585,153.862319,59.0
53440,97,2016-10-19,0,1,2016,10,19,2,�����������-1,292,...,87.106548,111.106047,82.545455,99.342857,99.437500,126.00,NaN,78.577465,104.589928,72.0


In [202]:
data.columns

Index([u'Count', u'Date', u'IsTrain', u'IsTest', u'Year', u'Month',
       u'MonthDay', u'WeekDay', u'Point', u'YearDay', u'ThroughMonth',
       u'ThroughWeek', u'ThroughDay', u'MeanForMonth', u'MeanForMonthDay',
       u'MeanForThroughMonth', u'MeanForThroughWeek', u'MeanForWeekDay',
       u'MeanForYear', u'MeanForYearDay', u'MeanForMonthByPoints',
       u'MeanForMonthDayByPoints', u'MeanForThroughMonthByPoints',
       u'MeanForThroughWeekByPoints', u'MeanForWeekDayByPoints',
       u'MeanForYearByPoints', u'MeanForYearDayByPoints'],
      dtype='object')

In [203]:
data.head()

,Count,Date,IsTrain,IsTest,Year,Month,MonthDay,WeekDay,Point,YearDay,...,MeanForWeekDay,MeanForYear,MeanForYearDay,MeanForMonthByPoints,MeanForMonthDayByPoints,MeanForThroughMonthByPoints,MeanForThroughWeekByPoints,MeanForWeekDayByPoints,MeanForYearByPoints,MeanForYearDayByPoints
0,8,2011-10-06,1,0,2011,10,6,3,���������-1,278,...,91.300923,85.111111,83.218391,267.076336,236.551724,32.333333,32.333333,212.893281,85.111111,220.4
1,88,2011-10-07,1,0,2011,10,7,4,���������-1,279,...,122.559729,85.111111,80.670455,267.076336,244.741379,32.333333,32.333333,290.403162,85.111111,230.8
2,1,2011-10-09,1,0,2011,10,9,6,���������-1,281,...,122.119897,85.111111,105.079545,267.076336,242.694915,32.333333,32.333333,266.624506,85.111111,204.2
3,1,2011-11-15,1,0,2011,11,15,1,���������-1,318,...,86.350693,85.111111,125.969697,258.991736,228.728814,1.000000,1.000000,194.561265,85.111111,208.2
4,2,2011-12-08,1,0,2011,12,8,3,���������-1,341,...,91.300923,85.111111,75.836538,235.299320,255.620690,95.652174,14.000000,212.893281,85.111111,182.6


In [204]:
data.shape

(53441, 27)

In [205]:
data.columns

Index([u'Count', u'Date', u'IsTrain', u'IsTest', u'Year', u'Month',
       u'MonthDay', u'WeekDay', u'Point', u'YearDay', u'ThroughMonth',
       u'ThroughWeek', u'ThroughDay', u'MeanForMonth', u'MeanForMonthDay',
       u'MeanForThroughMonth', u'MeanForThroughWeek', u'MeanForWeekDay',
       u'MeanForYear', u'MeanForYearDay', u'MeanForMonthByPoints',
       u'MeanForMonthDayByPoints', u'MeanForThroughMonthByPoints',
       u'MeanForThroughWeekByPoints', u'MeanForWeekDayByPoints',
       u'MeanForYearByPoints', u'MeanForYearDayByPoints'],
      dtype='object')

In [206]:
pd.isnull(data).sum() > 0

Count                          False
Date                           False
IsTrain                        False
IsTest                         False
Year                           False
Month                          False
MonthDay                       False
WeekDay                        False
Point                          False
YearDay                        False
ThroughMonth                   False
ThroughWeek                    False
ThroughDay                     False
MeanForMonth                   False
MeanForMonthDay                False
MeanForThroughMonth            False
MeanForThroughWeek              True
MeanForWeekDay                 False
MeanForYear                    False
MeanForYearDay                 False
MeanForMonthByPoints            True
MeanForMonthDayByPoints         True
MeanForThroughMonthByPoints     True
MeanForThroughWeekByPoints      True
MeanForWeekDayByPoints          True
MeanForYearByPoints             True
MeanForYearDayByPoints          True
d